# Supplementary Notebook for 'Anong Say ng Presidente Mo?'
## Using Twitter API

The following set of cells contains the Python implementation in accessing and retrieving data from Twitter API:

**Importing libraries**

In [15]:
import requests
import json
import getpass
import pandas as pd

**Storing personal bearer token**

In [ ]:
bearer_token = getpass.getpass()

**Retrieving user IDs by twitter handles**

In [ ]:
userids = requests.get(
    'https://api.twitter.com/2/users/by', 
    params={
        'usernames': 'lenirobredo,bongbongmarcos,IskoMoreno,SAPBongGo,MannyPacquiao,iampinglacson,leodymanggagawa'
    },
    headers={
        'Authorization': f'Bearer {bearer_token}'
    }
).json()

In [43]:
candidates = pd.json_normalize(userids['data'])
candidates

,id,name,username
0,47211842,Leni Robredo,lenirobredo
1,40204534,Bongbong Marcos,bongbongmarcos
2,998813499714224128,Isko Moreno Domagoso,IskoMoreno
3,1017117269975916544,Bong Go,SAPBongGo
4,254098644,Manny Pacquiao,MannyPacquiao
5,3007716060,PING LACSON,iampinglacson
6,1094854697595346946,Ka Leody de Guzman,LeodyManggagawa


**Getting all tweets of the targeted users**

In [78]:
df_tweets = pd.DataFrame()

for i in range(len(candidates)):
    #Loop through candidates
    user_id = candidates.iloc[i]['id']
    next_token = None
    
    # Get ten requests, 100 tweets each request
    for j in range(10):
        # First request, no next page token
        if j==0:
            tweets = requests.get(
                f'https://api.twitter.com/2/users/{user_id}/tweets', 
                params={
                    'exclude': 'retweets,replies',
                    'tweet.fields': 'public_metrics,created_at',
                    'max_results': 100
                },
                headers={
                    'Authorization': f'Bearer {bearer_token}'
                }
            ).json()
        # Succeeding requests, with next page token
        elif next_token is not None:
            tweets = requests.get(
                f'https://api.twitter.com/2/users/{user_id}/tweets', 
                params={
                    'exclude': 'retweets,replies',
                    'tweet.fields': 'public_metrics,created_at',
                    'max_results': 100,
                    'pagination_token': next_token
                },
                headers={
                    'Authorization': f'Bearer {bearer_token}'
                }
            ).json()
        else:
            continue
        
        try:
            next_token = tweets['meta']['next_token']
        except KeyError:
            next_token = None
        
        df_candidate_tweets = pd.json_normalize(data=tweets['data'])
        df_candidate_tweets['username'] = candidates.iloc[i]['username']

        if j==0 and i==0:
            df_tweets = df_candidate_tweets
        else:
            df_tweets = df_tweets.append(df_candidate_tweets, ignore_index=True)
        
df_tweets

,id,created_at,text,public_metrics.retweet_count,public_metrics.reply_count,public_metrics.like_count,public_metrics.quote_count,username
0,1462771275068182536,2021-11-22T13:13:43.000Z,The first shipment of Molnupiravir arrived in ...,490,25,4446,62,lenirobredo
1,1462771273537228802,2021-11-22T13:13:43.000Z,"In the agreement, the OVP will issue a guarant...",404,15,3900,7,lenirobredo
2,1462771270773194753,2021-11-22T13:13:42.000Z,"Molnupiravir, an oral pill for the treatment o...",8454,274,34237,1287,lenirobredo
3,1461314152924803079,2021-11-18T12:43:38.000Z,Dami na naman nangyayari 🔥 \n\nPero ito ang us...,4154,434,62592,763,lenirobredo
4,1460634783189331968,2021-11-16T15:44:04.000Z,There’s a new baby in the house and it was lov...,1838,320,38477,321,lenirobredo
...,...,...,...,...,...,...,...,...
4914,1095212099255422976,2019-02-12T06:44:47.000Z,Sundan ang campaign trail ni Ka Leody sa Talis...,4,0,16,0,LeodyManggagawa
4915,1095157845526405120,2019-02-12T03:09:12.000Z,Tuloy lang po tayo! #ManggagawaNaman ✊ https:/...,2,1,9,0,LeodyManggagawa
4916,1095157626713722880,2019-02-12T03:08:19.000Z,Maraming salamat po! #ManggagawaNaman ✊ https:...,0,0,4,0,LeodyManggagawa
4917,1095117782402977793,2019-02-12T00:30:00.000Z,Maaaring nang sundan ang kampanya ni Ka Leody ...,81,0,299,0,LeodyManggagawa


**Storing to a CSV file**

In [79]:
df_tweets.to_csv('candidate_tweets.csv')